In [3]:
import json
import time
import datetime

from episode_mining.winepi import WINEPI, WinEpiRules

In [4]:
def loadTimestampToEventDictFromFile(filePath):
    file = open(filePath)
    res = json.load(file)
    file.close()
    timestampToEventDict = {}
    keysSorted = sorted(res.keys())
    for key in keysSorted:
        timestamps = sorted(list(map(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"),res[key])))
        for ts in timestamps:
            eventList = timestampToEventDict.get(ts)
            if eventList is not None:
                eventList.append(key)
            else:
                newList = [key]
                timestampToEventDict[ts] = newList
    return timestampToEventDict

def getTimestampToIntegerDict1DayIncrement(timestampsSorted):
    firstTimestamp = timestampsSorted[0]
    lastTimestamp = timestampsSorted[-1]
    #SOCRATA SPECIFIC: TODO - change this for Wikipedia!
    incrementInDays=1
    curTimestamp = firstTimestamp
    timestampToIntegerDict = {}
    curInteger = 0
    while curTimestamp != None:
        timestampToIntegerDict[curTimestamp] = curInteger
        curInteger+=1
        if(curTimestamp == lastTimestamp):
            curTimestamp = None
        else:
            curTimestamp = curTimestamp + datetime.timedelta(days=incrementInDays)
    return timestampToIntegerDict

def getFinalSequence(timestampsSorted,timestampToIntegerDict,timestampToEventDict):
    finalList = []
    for ts in timestampsSorted:
        i = timestampToIntegerDict[ts]
        for key in timestampToEventDict[ts]:
            finalList.append((i,key))
    return finalList

#print(timestampToEventDict)
timestampToEventDict=loadTimestampToEventDictFromFile('/home/leon/Documents/lehre/ws2021/Master Project/data/column_table_changes_aggregated_grouped.json')
timestampsSorted = sorted(timestampToEventDict.keys())
timestampToIntegerDict = getTimestampToIntegerDict1DayIncrement(timestampsSorted)
sequence = getFinalSequence(timestampsSorted,timestampToIntegerDict,timestampToEventDict)

print("Starting WINEPI")
alg = WINEPI(sequence, episode_type='serial')
freqItems, suppData = alg.WinEpi(width=11, step=1, minFrequent=0.05)
print("Done with WINEPI")
winepiRules = WinEpiRules(freqItems, suppData, width=11, minConfidence=0.9)
ruleList = winepiRules.generateRules()
winepiRules.printRules(ruleList)
# 0 Alice 24
# 1 Bob 50
# 2 Charlie 18




Starting WINEPI


KeyboardInterrupt: 

In [ ]:
sequence = sorted([
    (31, 'E'), (31, 'D'), (33, 'F'), (35, 'A'), (37, 'B'), (38, 'C'), (39, 'E'),
    (40, 'F'), (42, 'C'), (44, 'D'), (46, 'B'), (47, 'A'), (48, 'D'), (50, 'C'),
    (53, 'E'), (54, 'F'), (55, 'C'), (57, 'B'), (58, 'E'), (59, 'A'), (60, 'E'),
    (61, 'C'), (62, 'F'), (65, 'A'), (67, 'D'),
], key=lambda x:x[0])

sequence = sorted([
    (31, 'E'), (31, 'D'), (33, 'E'), (33, 'D'), (33, 'E'), (58, 'E'), (58, 'D')
], key=lambda x:x[0])

In [ ]:
alg = WINEPI(sequence, episode_type='parallel')
freqItems, suppData = alg.WinEpi(width=5, step=1, minFrequent=0.1)

In [ ]:
winepiRules = WinEpiRules(freqItems, suppData, width=5, minConfidence=0.7)
ruleList = winepiRules.generateRules()
winepiRules.printRules(ruleList)